In [ ]:
import pygmt
import pandas as pd
import numpy as np
import obspy
from pyproj import Proj
import matplotlib.pyplot as plt

In [ ]:
# 2D no stat corr
# D2nsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/2DCat/bremUp_2Dnsc.cat')
D2nsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/2DCat/bremUp_2DnscWids.cat')


# 2D w/ Station Corrections
# D2sc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/2DCat/bremUp_2Dsc.cat')
D2sc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/2DCat/bremUp_2DscWids.cat')


# 3D no stat corr
# D3nsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/3DCat/bremUp_3Dnsc.cat')
D3nsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/3DCat/bremUp_3DnscWids.cat')


# 3D w/ Station Corrections
# D3sc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/3DCat/bremUp_3Dsc.cat')
D3sc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/3DCat/bremUp_3DscWids.cat')


#cvm no stat corr
# CVMnsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/cvmCat/bremUp_CVMnsc.cat')
CVMnsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/cvmCat/bremUp_CVMnscWids.cat')


# cvm w/ Station Corrections 
# CVMsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/cvmCat/bremUp_CVMsc.cat')
CVMsc = pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/cvmCat/bremUp_CVMscWids.cat')

# Hypoinverse:
cat = pd.read_csv('/Users/banjo/Projects/nll_pnsn/bremerton/bremUpdated.cat')

# Reid Merrill's relocations 
MVM=pd.read_csv('/Users/banjo/Projects/nll_pnsn/util/catalogs/bremUp_MVM.cat')

# event_id_s for locations in nlloc 
# event_ids=pd.read_csv('/Users/banjo/Projects/nll_pnsn/NLLoc/test/brem2D/loc/event_ids')

In [ ]:
p = Proj(proj='utm' ,zone=10 ,ellps='WGS84' ,datum='WGS84' ,units='m')

In [ ]:
def plot_xsec_comp(df1,df2,color1,color2,label1,label2,save):
    
    ''' 
        Function to plot the locations at depth with a NS xsection line and 
        a EW xsection line. 
        EW xsection line is 1 degree 
        NS xsection liune is 0.5 degree. 
        Input: 
            df1: pandas dataframe the catalog 1 of interest
            df2: pandas dataframe  the 2nd catalog of interest
            color1: string matplot lib color for plotting
            color2: string matplot lib color for plotting
            label1: string name of df1
            label2: string name of df2
            save: 1 if you want to save it, 0 if you dont. 
        Returns 3 plots of lat lon and dep 
    '''
    fig = pygmt.Figure()
    pygmt.config(MAP_FRAME_TYPE="plain")
    pygmt.config(MAP_FRAME_AXES='NesW')
    pygmt.config(FORMAT_GEO_MAP="ddd.x")


    ##
    ### Top Left ###
    ##
    r1 = [-123.0,-122.0,0,55.55]
    fig.basemap(region=r1,frame="a10.0f -JX4i/-2i")

    fig.plot(x = df1['mlon'].values, y = df1['mdep'].values,
         style = 'c0.08c',color = color1,pen = 'black',label= label1)

    fig.shift_origin(xshift= "4.5i")
    ##
    ### Top Right ##
    ##
    r1 = [-123.0,-122.0,0,55.55] 
    fig.basemap(region=r1,frame="a10.0f -JX4i/-2i")

    fig.plot(x = df2['mlon'].values, y = df2['mdep'].values,
         style = 'c0.08c',color = color2 ,pen = 'black',label=label2)

    fig.legend()

    fig.shift_origin(xshift= "-4.5i",yshift= "-2.25i")

    ##
    ### Bottom Left ##
    ##

    r3 = [47.25,47.750,0,55.55]

    fig.basemap(region=r3,frame="a10.0f -JX2i/-2i -BNseW")

    fig.plot(x = df1['mlat'].values, y = df1['mdep'].values,
         style = 'c0.08c',color = color1 ,pen = 'black',label=label1)

    fig.shift_origin(xshift= "4.5i")

    ##
    ##Bottom Right ##
    ##
    r3 = [47.250,47.750,0,55.55]
    fig.basemap(region=r3,frame="a10.0f -JX2i/-2i -BNseW")

    fig.plot(x = df2['mlat'].values, y = df2['mdep'].values,
         style = 'c0.08c',color = color2 ,pen = 'black',label=label2)

#     fig.show()
    if save == 1:
        fig.savefig('/Users/banjo/Desktop/bremComp/' + label1 +'_vs_'+ label2+'.png')
    else:
        fig.show()
        
    return fig

In [ ]:
def diff_plot(df1,df2,color,name1,name2,save):
    ''' 
    Function to plot the differences in Lat, Lon and Depth
    Input: 
        df1: pandas dataframe the catalog 1 of interest
        df2: pandas dataframe  the 2nd catalog of interest
        color: string matplot lib color for plotting
        name1: string name of df1
        name2: string name of df2
    Returns 3 plots of lat lon and dep 
    '''

    df1mlat = df1['mlat'].values
    df2mlat = df2['mlat'].values
    df1Lon = df1['mlon'].values
    df2Lon = df2['mlon'].values

    df1Coord = [df1Lon,df1mlat]
    df2Coord = [df2Lon,df2mlat]

    df1utm = p(df1Coord[0],df1Coord[1])
    df1utme=df1utm[0]
    df1utmn=df1utm[1]


    df2utm = p(df2Coord[0],df2Coord[1])
    df2utme=df2utm[0]
    df2utmn=df2utm[1]

    dx = df1utme - df2utme
    dy = df1utmn - df2utmn

    df1dep = df1['mdep'].values
    df2dep = df2['mdep'].values
    ddep = df1dep - df2dep
    ddep = ddep*1000
    
    #plot 1:
    plt.figure(figsize=(10,15))
    plt.subplot(3, 1, 1)
    markerline, stemlines, baseline = plt.stem(
        range(len(df1['evids'].values)),dx, linefmt='grey', markerfmt='o', bottom=0,basefmt='black')
    markerline.set_markerfacecolor('white')
    markerline.set_markeredgecolor(color)
    plt.ylabel('difference (m)')
    plt.ylim([-5000.,5000.])
    plt.xlabel('events')
    plt.title('Latitude: '+ str(name1) +' minus ' +str(name2))
    plt.grid()
    plt.show()

    plt.figure(figsize=(10,15))
    plt.subplot(3, 1, 2)
    markerline, stemlines, baseline = plt.stem(
        range(len(df1['evids'].values)),dy, linefmt='grey', markerfmt='o', bottom=0,basefmt='black')
    markerline.set_markerfacecolor('white')
    markerline.set_markeredgecolor(color)
    plt.ylabel('difference (m)')
    plt.ylim([-5000.,5000.])
    plt.xlabel('events')
    plt.title('Longitude: '+ str(name1) +' minus ' +str(name2))
    plt.grid()
    plt.show()

    #plot 3:
    plt.figure(figsize=(10,15))
    plt.subplot(3, 1, 3)
    markerline, stemlines, baseline = plt.stem(
        range(len(df1['evids'].values)),ddep, linefmt='grey', markerfmt='o', bottom=0,basefmt='black')
    markerline.set_markerfacecolor('white')
    markerline.set_markeredgecolor(color)
    plt.ylabel('difference (m)')
    plt.ylim([-5000.,5000.])
    plt.xlabel('events')
    plt.title('Depth: '+ str(name1) +' minus ' +str(name2))
    plt.grid()
    plt.show()
    
    if save == 1:
        plt.savefig('/Users/banjo/Desktop/bremComp/' + label1 +'_minus_'+ label2+'.png')
    else:
        plt.show()

In [ ]:
diff_plot(D2nsc,D3nsc,'seagreen','D2nsc','D3nsc',0)
# diff_plot(cat,CVMnsc,'navy','PNSN','CVMnsc',0)
# diff_plot(cat,CVMsc,'black','PNSN','CVMsc',0)
# diff_plot(cat,D2nsc,'firebrick','PNSN','D2nsc',0)
# diff_plot(cat,D3nsc,'purple','PNSN','D3nsc',0)

In [ ]:
# diff_plot(D2sc,D3sc,'seagreen','D2sc','D3sc',0)
# plot_xsec_comp(D2sc,D3sc,'seagreen','firebrick','D2sc','D3sc',0)


# diff_plot(D2nsc,D3nsc,'seagreen','D2nsc','D3nsc',0)
plot_xsec_comp(D2nsc,D3nsc,'dodgerblue','indianred','D2nsc','D3nsc',0)

# diff_plot(cat,D2nsc,'firebrick','PNSN','D2nsc',0)
# plot_xsec_comp(cat,D2nsc,'dodgerblue','mediumpurple1','PNSN','D2nsc',0)


# # diff_plot(cat,CVMsc,'black','PNSN','CVMsc',0)
# plot_xsec_comp(CVMsc,D3sc,'coral','firebrick','CVMsc','D3sc',0)


# # diff_plot(cat,CVMnsc,'black','PNSN','CVMnsc',0)
# plot_xsec_comp(cat,CVMsc,'coral','mediumpurple1','PNSN','CVMsc',0)

# plot_xsec_comp(cat,D2nsc,'seagreen','firebrick','PNSN','D2nsc',0)




# # diff_plot(D2nsc,D3nsc,'seagreen','D2nsc','D3nsc',0)
# # diff_plot(CVMnsc,cat,'navy','CVMnsc','PNSN',0)
# # diff_plot(cat,CVMsc,'black','PNSN','CVMsc',0)
# # diff_plot(cat,D2nsc,'firebrick','PNSN','D2nsc',0)
# diff_plot(cat,D3nsc,'purple','PNSN','D3nsc',0)

In [ ]:
diff_plot(D2nsc,D3nsc,'seagreen','D2nsc','D3nsc',0)
diff_plot(CVMnsc,cat,'navy','CVMnsc','Hypoinverse',0)
diff_plot(cat,CVMsc,'black','Hypoinverse','CVMsc',0)
diff_plot(cat,D2nsc,'firebrick','Hypoinverse','D2nsc',0)
diff_plot(cat,D3nsc,'purple','Hypo','D3nsc',0)